# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
#import pandas as pd
import cassandra
#import re
import os
import glob
#import numpy as np
#import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#get total number of rows 
#print(len(full_data_rows_list))
#check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
                        CREATE KEYSPACE IF NOT EXISTS sparkify
                        WITH REPLICATION = 
                        {'class': 'SimpleStrategy', 'replication_factor': 1}
                    """
)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338 and itemInSession = 4

    select artist, songTitle, songLen
    from music_session
    where sessionId = 338
    and itemInSession = 4                    

### Table structure:

* The **primary key** will be represented by a **composite partitioning key** comprised of **sessionId and itemInSession**. These two columns serve as filtering columns so it's important that they are both part of the partitioning key. These two columns will also provide **uniqueness** to rows inside the table.
* There will be **_no clustering columns_** as there's no need for ordering data by a specific column.
* The **non-key columns** that hold information relevant to the query will be: **artist, songLen, songTitle**.

In [8]:
# Create table statement
query = """
            CREATE TABLE IF NOT EXISTS song_details_in_session (
                sessionId int,
                itemInSession int,     
                artist text,
                songLen float,
                songTitle text,
                PRIMARY KEY ((sessionId, itemInSession))
            )
        """

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

# Read file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    # Read line by line
    for line in csvreader:
        #print(line)
# Assign the INSERT statements into the `query` variable
        query = """
                    INSERT INTO song_details_in_session (
                        sessionId,
                        itemInSession,     
                        artist,
                        songLen,
                        songTitle
                        )
                    VALUES (%s, %s, %s, %s, %s)
                """
        # Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (
                                    int(line[8]),
                                    int(line[3]),
                                    line[0],
                                    float(line[5]), 
                                    line[9]
                                   )
                           )
        except Exception as e:
            print(e)

In [10]:
# Verify the data was entered into the table
query = """
            select artist, songTitle, songLen
            from song_details_in_session
            where sessionId = 338
            and itemInSession = 4
        """ 

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.songtitle, row.songlen)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

    select artist, songTitle, userFirstName, userLastName
    from music_session2
    where userid = 10 
    and sessionid = 182
    order by itemInSession

### Table structure:

* The **primary key** will be represented by a **composite partitioning key** comprised of **sessionId and userId**. These two columns serve as filtering columns so it's important that they are both part of the partitioning key. These two columns will also provide **uniqueness** to rows inside the table.
* Part of the primary is also a **clustering column, itemInSession**, which is needed in order to sort de data.
* The **non-key columns** that hold information relevant to the query will be: **artist, songTitle, userFirstName, userLastName**.

In [11]:
# Create table statement
query = """
            CREATE TABLE IF NOT EXISTS songs_playlist_in_session (
                sessionId int,
                userId int,    
                itemInSession int,                
                userFirstName text,
                userLastName text,
                artist text,                
                songTitle text,
                PRIMARY KEY ((sessionId, userId), itemInSession)
            )
        """

try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

# Read file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    # Read line by line
    for line in csvreader:
        #print(line)
# Assign the INSERT statements into the `query` variable
        query = """
                    INSERT INTO songs_playlist_in_session (
                        sessionId,
                        userId,    
                        itemInSession,                
                        userFirstName,
                        userLastName,
                        artist,                
                        songTitle
                        )
                    VALUES (%s, %s, %s, %s, %s, %s, %s)
                """
        # Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (
                                    int(line[8]),
                                    int(line[10]),
                                    int(line[3]),                
                                    line[1],
                                    line[4],
                                    line[0],                 
                                    line[9]
                                   )
                           )
        except Exception as e:
            print(e)

In [13]:
# Verify that the data was entered into the table
query = """
            select artist, songTitle, userFirstName, userLastName, itemInSession
            from songs_playlist_in_session
            where userid = 10 
            and sessionid = 182
        """ 

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.iteminsession, row.artist, row.songtitle, row.userfirstname, row.userlastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

    select userFirstName, userLastName
    from music_session3
    where song = 'All Hands Against His Own'                 

### Table structure:

* The **primary key** will be represented by a **composite partitioning key** comprised of **songTitle and userId**. songTitle serves as a filtering column while userId will be used alongside songTitle in order to provide **uniqueness** to rows inside the table. 
* There will be **_no clustering columns_** as there's no need for ordering data by a specific column.
* The **non-key columns** that hold information relevant to the query will be: **userFirstName, userLastName**.

In [14]:
# Create table statement
query = """
            CREATE TABLE IF NOT EXISTS users_listening_to_song (
                songTitle text,                                
                userId int,
                userFirstName text,
                userLastName text,
                PRIMARY KEY ((songTitle, userId))
            )
        """

try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

# Read file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    # Read line by line
    for line in csvreader:
        #print(line)
# Assign the INSERT statements into the `query` variable
        query = """
                    INSERT INTO users_listening_to_song (
                        songTitle,                                        
                        userId,
                        userFirstName,
                        userLastName                  
                        )
                    VALUES (%s, %s, %s, %s)
                """
        # Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (
                                    line[9],                                    
                                    int(line[10]),
                                    line[1],
                                    line[4],
                                   )
                           )
        except Exception as e:
            print(e)

In [16]:
# Verify the data was entered into the table
query = """
            select userFirstName, userLastName
            from users_listening_to_song
            where songTitle = 'All Hands Against His Own'
            ALLOW FILTERING
        """ 
#the ALLOW FILTERING clause is used in order to bypass the execution error that prevents the query from returning results.

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.userfirstname, row.userlastname)

Tegan Levine
Sara Johnson
Jacqueline Lynch


### Drop the tables before closing out the sessions

In [17]:
# Drop the tables before closing out the sessions
for tbl in ('song_details_in_session', 'songs_playlist_in_session', 'users_listening_to_song'):
    try:
        session.execute('drop table ' + tbl)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()